In [2]:
import pandas as pd
import re,glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

from tqdm import tqdm
import seaborn as sns

# Preprocessing Data

Rain fall lv.

In [3]:
def rename(path_name):
    path_name = str(re.sub('(rain2007-2020/|water-level2007-2020/)','',path_name[38:]))
    path_name = re.sub('.csv','',path_name)
    return path_name

def df_process(file):
    try:
        df = pd.read_csv(file,parse_dates=[['date', 'time']])
    except:
        df = pd.read_csv(file,parse_dates=['date'])
    df['station'] = rename(file)
    return df

def open_concat_csv(folder_path):
    all_filenames = [i for i in glob.glob(os.path.join(folder_path, '*.csv'))]
    df = pd.concat([df_process(f) for f in all_filenames])
    return df

path_rain = "../hii-telemetering-batch-data-master/rain2007-2020/"


In [4]:
rain_df = open_concat_csv(path_rain)
# replace -999 value to NaN
rain_df['rain'] = rain_df["rain"].replace({-999:np.nan})

print(rain_df.shape)
rain_df.head()

(4601312, 3)


,date,rain,station
0,2007-01-01,NaN,SWR028
1,2007-01-02,NaN,SWR028
2,2007-01-03,NaN,SWR028
3,2007-01-04,NaN,SWR028
4,2007-01-05,NaN,SWR028


In [5]:
rain_df_st = pd.DataFrame()
for name,group in rain_df.groupby('station'):
    if rain_df_st.empty:
        rain_df_st = group.set_index("date")[["rain"]].rename(columns={'rain':name})
    else:
        rain_df_st = rain_df_st.join(group.set_index("date")[["rain"]].rename(columns={'rain':name}))

print(rain_df_st.shape)
rain_df_st.head()

(4961, 929)


,ACRU,ANLI,BAKI,BARI,BBHN,BBON,BBUA,BBWN,BCAP,BCNG,...,YOM003,YOM004,YOM005,YOM006,YOM007,YOM008,YOM009,YOM010,YOM011,YOM012
date,,,,,,,,,,,,,,,,,,,,,
2007-01-01,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-02,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-03,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-04,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-05,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN,0.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


WATER Level data

In [6]:
path_water = "../hii-telemetering-batch-data-master/water-level2007-2020/"
water_df = open_concat_csv(path_water)

print(water_df.shape)
print(water_df.columns)

(225604800, 3)
Index(['date_time', 'wl', 'station'], dtype='object')


In [9]:
water_df['wl'] = water_df['wl'].replace({-999:np.nan})
water_df.head()

,date_time,wl,station
0,2007-01-01 00:00:00,NaN,TBW018
1,2007-01-01 00:10:00,NaN,TBW018
2,2007-01-01 00:20:00,NaN,TBW018
3,2007-01-01 00:30:00,NaN,TBW018
4,2007-01-01 00:40:00,NaN,TBW018


In [10]:
df_wl_st = pd.DataFrame()

for name,group in water_df.groupby('station'):
    if df_wl_st.empty:
        df_wl_st = group.set_index(["date_time"])[["wl"]].rename(columns={'wl':name})
    else:
        df_wl_st = df_wl_st.join(group.set_index(["date_time"])[["wl"]].rename(columns={'wl':name}))

print(df_wl_st.shape)     
df_wl_st.head()

(714384, 316)


,BKK001,BKK002,BKK003,BKK004,BKK005,BKK006,BKK007,BKK008,BKK009,BKK012,...,YOM003,YOM004,YOM005,YOM006,YOM007,YOM008,YOM009,YOM010,YOM011,YOM012
date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [11]:
#rain_df_st.to_csv("data/rain.csv")
#df_wl_st.to_csv('data/water.csv')

# Selected related station only

In [34]:
filepath = ("../hii-telemetering-batch-data-master/")
water_st = pd.read_csv(filepath+"station_metadata-water-level.csv")
water_st = water_st.loc[(water_st['basin']=="แม่น้ำปิง")|(water_st['basin']=="แม่น้ำเจ้าพระยา")|(water_st['basin']=="แม่น้ำน่าน")]
#น้ำเหนือที่ส่งผลให้ภาคกลางเกิดน้ำท่วมคือ แม่น้ำปิง กับ น่าน เพราะฉะนั้นโฟกัสแค่สองแม่น้ำนี้เป็นหลัก


In [65]:
def intersection(lst1, lst2): 
    return list(set(lst1) & set(lst2)) 
col = [i for i in water_st['code']]
col = intersection(col,df_wl_st.columns)


In [70]:
print(df_wl_st[col].shape)
df_wl_st[col].head()

(714384, 72)


,THA001,CPY004,PIN005,CPY012,NAN007,TB0004,BKK007,CHM005,CPY015,NAN011,...,PIN006,BKK018,TB0001,BKK021,NAN003,PIN002,BKK005,NAN009,TB0002,NAN012
date_time,,,,,,,,,,,,,,,,,,,,,
2007-01-01 00:00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55.09,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:10:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:20:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:30:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2007-01-01 00:40:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


To do list
เหลือต้องมาดูว่า สถานีฝน เอาอะไรไม่เอาอะไรบ้าง
หลังจากนั้นค่อยมาดูว่า เลือกปีไหนได้บ้าง อันไหน nan เยอะเกิน
แล้วค่อยยัดลง LSTM